<a href="https://colab.research.google.com/github/spannozzo/job-resume-matching-algo/blob/master/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/ruozhengu/job-resume-matching-algo

Cloning into 'job-resume-matching-algo'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 28 (delta 4), reused 10 (delta 0), pack-reused 0
Unpacking objects: 100% (28/28), 17.24 KiB | 1.57 MiB/s, done.


In [25]:
# import packages
import bs4
from bs4 import BeautifulSoup
import requests


# get soup object
def get_soup(text):
	return BeautifulSoup(text, "lxml", from_encoding="utf-8")


# extract company
def extract_company(div):
    company = div.find_all(name="span", attrs={"class":"company"})
    if len(company) > 0:
        for b in company:
            return (b.text.strip())
    else:
        sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
        for span in sec_try:
            return (span.text.strip())
    return 'NOT_FOUND'


# extract job salary
def extract_salary(div):
    try:
        return (div.find('nobr').text)
    except:
        try:
            div_two = div.find(name='span', attrs={'class':'salaryText'})
            div_three = div_two.find('div')
            salaries.append(div_three.text.strip())
        except:
            return ('NOT_FOUND')
    return 'NOT_FOUND'


# extract job location
def extract_location(div):
    for span in div.findAll('div', attrs={'class': 'location'}):
        return (span.text)
    return 'NOT_FOUND'


# extract job title
def extract_job_title(div):
    for a in div.find_all(name='a', attrs={'data-tn-element':'jobTitle'}):
        return (a['title'])
    return('NOT_FOUND')


# extract jd summary
def extract_summary(div):
    spans = div.findAll('div', attrs={'class': 'summary'})
    for span in spans:
        return (span.text.strip())
    return 'NOT_FOUND'


# extract link of job description
def extract_link(div):
    for a in div.find_all(name='a', attrs={'data-tn-element':'jobTitle'}):
        return (a['href'])
    return('NOT_FOUND')


# extract date of job when it was posted
def extract_date(div):
    try:
        spans = div.findAll('span', attrs={'class': 'date'})
        for span in spans:
            return (span.text.strip())
    except:
        return 'NOT_FOUND'
    return 'NOT_FOUND'


# extract full job description from link
def extract_fulltext(url):
    try:
        page = requests.get('http://www.indeed.com' + url)
        soup = BeautifulSoup(page.text, "lxml", from_encoding="utf-8")
        spans = soup.findAll('div', attrs={'class': 'jobsearch-jobDescriptionText'})
        for span in spans:
            return (span.text.strip())
    except:
        return 'NOT_FOUND'
    return 'NOT_FOUND'


# write logs to file
def write_logs(text):
    # print(text + '\n')
    f = open('log.txt','a')
    f.write(text + '\n')
    f.close()


In [26]:
# import packages
import requests
import pandas as pd
import time


# limit per sity
max_results_per_city = 990

# db of city
#city_set = ['Toronto', 'Markham', 'North+York', 'Scarbough', 'Missisauga', 'Richmond+Hill']
city_set = ['Toronto', 'Vancouver', 'alberta', 'waterloo', 'london']
# job roles
job_set = ['all']


df = pd.DataFrame(columns = ['unique_id', 'job_title', 'company_name', 'location', 'full_text'])


# loop on all cities
for city in city_set:

    print("Start searching city: " + city)
    cnt = 0

    # for results
    for start in range(0, max_results_per_city, 10):

        # get dom
        page = requests.get('http://ca.indeed.com/jobs?q=&l=' + str(city) + '&sort=date&start=' + str(start))

        #ensuring at least 1 second between page grabs
        time.sleep(1)

        #fetch data
        soup = get_soup(page.text)
        divs = soup.find_all(name="div", attrs={"class":"row"})

        # if results exist
        if(len(divs) == 0):
            break

        # for all jobs on a page
        for div in divs:

            cnt += 1

            #specifying row num for index of job posting in dataframe
            num = (len(df) + 1)

            #job data after parsing
            job_post = []

            #append unique id
            job_post.append(div['id'])

            #grabbing job title
            job_post.append(extract_job_title(div))

            #grabbing company
            job_post.append(extract_company(div))

            #grabbing location name
            job_post.append(extract_location(div))

            #grabbing link
            link = extract_link(div)

            #grabbing full_text
            job_post.append(extract_fulltext(link))

            #appending list of job post info to dataframe at index num
            df.loc[num] = job_post

    print(city + " is completed with count as " + str(cnt))

#saving df as a local csv file
df.to_csv('jobs.csv', encoding='utf-8')



Start searching city: Toronto


/usr/local/lib/python3.10/dist-packages/bs4/__init__.py:226: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


Toronto is completed with count as 0
Start searching city: Vancouver
Vancouver is completed with count as 0
Start searching city: alberta
alberta is completed with count as 0
Start searching city: waterloo
waterloo is completed with count as 0
Start searching city: london
london is completed with count as 0
